In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
import json
import requests
import os
from datetime import datetime, timedelta
from xgboost import XGBClassifier # Needed for loading XGBoost
from includes.technical_indicators import *
from includes.helpers import get_crypto_data


2025-12-15 20:56:58.957635: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:

# --- 1. Define Indicator Functions (Must be available to the class) ---
# (Ensure your calculate_rsi, calculate_macd, etc., are defined here or imported)

class CryptoPredictor:
    def __init__(self, prefix="btc_lstm"):
        self.prefix = prefix
        self.model = None
        self.scaler = None
        self.threshold = 0.5
        self.window_size = 10 
        self.model_type = "keras"
        
    def load_artifacts(self):
        # """Smart loader: Detects if model is Keras, XGBoost, or Pickle."""
        # print(f"Loading system for prefix: {self.prefix}...")
        
        folder = "saved_artifacts"
        path_keras = f"{folder}/{self.prefix}_model.keras"
        path_json  = f"{folder}/{self.prefix}_model.json"
        path_pkl   = f"{folder}/{self.prefix}_model.pkl"
        
        if os.path.exists(path_keras):
            self.model = tf.keras.models.load_model(path_keras)
            self.model_type = "keras"
        elif os.path.exists(path_json):
            self.model = XGBClassifier()
            self.model.load_model(path_json)
            self.model_type = "xgboost"
        elif os.path.exists(path_pkl):
            with open(path_pkl, "rb") as f:
                self.model = pickle.load(f)
            self.model_type = "sklearn"
        else:
            raise ValueError(f"No model found for prefix '{self.prefix}'")

        with open(f"{folder}/{self.prefix}_scaler.pkl", "rb") as f:
            self.scaler = pickle.load(f)
            
        with open(f"{folder}/{self.prefix}_config.json", "r") as f:
            config = json.load(f)
            self.threshold = config["optimal_threshold"]

    def fetch_recent_data(self, ticker="BTC-USD", target_date_str=None):
        """
        Universal Data Fetcher using 'requests'.
        Bypasses 'ImpersonateError' from yfinance.
        """
        if target_date_str:
            target_dt = datetime.strptime(target_date_str, "%Y-%m-%d")
        else:
            target_dt = datetime.now()

        # End: 23:59:59 of target date
        end_dt = target_dt.replace(hour=23, minute=59, second=59)
        period2 = int(end_dt.timestamp())
        
        # Start: 180 days lookback (plenty for indicators)
        start_dt = end_dt - timedelta(days=180)
        period1 = int(start_dt.timestamp())

        # Direct Yahoo JSON API URL
        url = f"https://query1.finance.yahoo.com/v8/finance/chart/{ticker}"
        params = {
            "period1": period1,
            "period2": period2,
            "interval": "1d",
            "events": "div,split"
        }
        headers = {'User-Agent': 'Mozilla/5.0'}

        try:
            response = requests.get(url, params=params, headers=headers, timeout=10)
            response.raise_for_status()
            data = response.json()
            
            result = data['chart']['result'][0]
            timestamps = result['timestamp']
            quote = result['indicators']['quote'][0]
            
            df = pd.DataFrame({
                'Date': pd.to_datetime(timestamps, unit='s'),
                'Open': quote['open'],
                'High': quote['high'],
                'Low': quote['low'],
                'Close': quote['close'],
                'Volume': quote['volume']
            })
            df.set_index('Date', inplace=True)
            # Normalize index to remove time (midnight)
            df.index = df.index.normalize() 
            df = df.dropna()
            
            if df.empty:
                raise ValueError(f"Data for {ticker} is empty.")
                
            return df
            
        except Exception as e:
            # We raise the error now so you know downloading failed
            raise ValueError(f"❌ FAILED to download {ticker}: {e}")

    def preprocess_live_data(self, df):
        """Calculates indicators and fetches macros using the working API logic."""
        # 1. Calculate Technical Indicators
        df['RSI'] = calculate_rsi(df)
        df['MACD_Line'], df['Signal_Line'] = calculate_macd(df)
        df['Williams_R'] = calculate_williams_r(df)
        df['Bollinger_B'] = calculate_bollinger_b(df)
        df['NATR'] = calculate_natr(df)
        df['Vol_ROC'] = calculate_volume_roc(df)
        df['Vol_Ratio'] = calculate_vol_ratio(df, period=14)

        # 2. --- CORRECTED MACRO FETCHING ---
        # Get the latest date from the current BTC dataframe
        last_date_str = df.index[-1].strftime('%Y-%m-%d')
        
        # Map: Feature Name -> Ticker Symbol
        macro_map = {
            'DXI': 'DX-Y.NYB',
            'SP500': '^GSPC',
            'GOLD': 'GC=F',
            'Ten_Y': '^TNX'
        }

        # print(f"   Fetching macro data for {last_date_str}...")

        for col_name, ticker in macro_map.items():
            # Use self.fetch_recent_data (which uses requests)
            # This ensures we get real data, or crash if impossible (no zeros!)
            macro_df = self.fetch_recent_data(ticker, target_date_str=last_date_str)
            
            # Extract only the Close column and rename it
            macro_series = macro_df['Close'].rename(col_name)
            
            # Merge into BTC dataframe
            df = df.join(macro_series, how='left')
            
            # Forward Fill: Essential because stocks don't trade on weekends
            df[col_name] = df[col_name].ffill()
            
            # Backward Fill: Catch any tiny gaps at the start of the window
            df[col_name] = df[col_name].bfill()

        # 3. Drop NaNs (Should only happen if macro fetch completely failed/returned nothing)
        df = df.dropna()
        
        # 4. Feature Selection
        feature_cols =['Open', 'High', 'Low', 'Close', 'Volume', 
                       'DXI', 'SP500', 'GOLD', 'Ten_Y', 
                       'RSI', 'MACD_Line', 'Signal_Line', 'Williams_R', 
                       'Bollinger_B', 'NATR', 'Vol_ROC', 'Vol_Ratio']
        
        # Double check we have all columns (Safety)
        missing_cols = [c for c in feature_cols if c not in df.columns]
        if missing_cols:
            raise ValueError(f"Missing columns after processing: {missing_cols}")

        # 5. Get the LAST Window
        last_window = df[feature_cols].tail(self.window_size).values
        
        if len(last_window) < self.window_size:
            raise ValueError(f"Not enough data. Needed {self.window_size}, got {len(last_window)}.")
            
        # 6. Scale
        scaled_window = self.scaler.transform(last_window)
        
        # 7. Reshape
        if self.model_type == "keras":
            final_input = np.expand_dims(scaled_window, axis=0)
        else:
            final_input = scaled_window.flatten().reshape(1, -1)
        
        return final_input, df['Close'].iloc[-1], df.index[-1]

    def predict_next_day(self, target_date):
        """Fetches data, predicts, and returns stats."""
        # 1. Fetch BTC Data
        df = self.fetch_recent_data("BTC-USD", target_date_str=target_date)
        
        # 2. Process (will now fetch macros properly)
        X_input, current_price, _ = self.preprocess_live_data(df)
        
        # 3. Predict
        if self.model_type == "keras":
            prob = self.model.predict(X_input, verbose=0)[0][0]
        else:
            prob = self.model.predict_proba(X_input)[0][1]
        
        return {
            "Model": self.prefix.replace("btc_", "").upper(),
            "Probability": prob, 
            "Threshold": self.threshold,
            "Trend": "UP 🟢" if prob > self.threshold else "DOWN 🔴",
            "Confidence": abs(prob - 0.5) * 200
        }

In [5]:
import pandas as pd

model_prefixes = ["btc_lstm", "btc_xgboost", "btc_rf", "btc_svm", "btc_knn"]
target_date = "2025-12-13"

print(f"Data Date:       {target_date}")
print("-" * 60)

# 1. Collect results in a list
results_list = []

for prefix in model_prefixes:
    try:
        bot = CryptoPredictor(prefix)
        bot.load_artifacts()
        
        # Get dictionary return
        stats = bot.predict_next_day(target_date)
        results_list.append(stats)
        
    except Exception as e:
        print(f"⚠️ Error with {prefix}: {e}")
        results_list.append({"Model": prefix, "Trend": "ERROR"})

# 2. Create DataFrame for the table
if results_list:
    df = pd.DataFrame(results_list)

    # 3. Format numbers for cleaner display (Optional)
    # Probability: 0.55 -> 55.00%
    if 'Probability' in df.columns:
        df['Probability'] = df['Probability'].apply(lambda x: f"{x:.2%}" if isinstance(x, (float, int)) else x)
    
    # Confidence: 10.543 -> 10.54%
    if 'Confidence' in df.columns:
        df['Confidence'] = df['Confidence'].apply(lambda x: f"{x:.2f}%" if isinstance(x, (float, int)) else x)

    # Threshold: 0.5 -> 0.50
    if 'Threshold' in df.columns:
        df['Threshold'] = df['Threshold'].apply(lambda x: f"{x:.2f}" if isinstance(x, (float, int)) else x)

    # 4. Print Table (hide index number for cleaner look)
    print("\n--- 🏆 MODEL CONSENSUS TABLE ---")
    print(df.to_string(index=False))

Data Date:       2025-12-13
------------------------------------------------------------

--- 🏆 MODEL CONSENSUS TABLE ---
  Model Probability Threshold  Trend Confidence
   LSTM      63.38%      0.38   UP 🟢     26.75%
XGBOOST      50.10%      0.51 DOWN 🔴      0.19%
     RF      51.11%      0.48   UP 🟢      2.22%
    SVM      51.70%      0.52 DOWN 🔴      3.41%
    KNN      50.08%      0.45   UP 🟢      0.15%
